# DO NACs (HPC version)

This file demonstrates how to run the calculations of the NACs in the KS space, using QE.

In particular, this example is designed to run calculations on UB HPC cluster, CCR (Center for Computational Research). More specifically, using the nodes of the Akimov group (*valhalla* cluster).

Unline for the on-laptop version, we may need to wait until the submitted jobs are done, so we wil not be able to run the plotting and other data analysis calculations right away. So those parts are removed.

So, lets start by loading the required modules.

In [35]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
    

from libra_py import hpc_utils
from libra_py import data_read
from libra_py import data_outs
from libra_py import units
from libra_py import QE_methods
from libra_py.workflows.nbra import step2

For convenience, lets print out the location of the current working directory.

This directory should contain a folder called **PP**, in which we should have placed the atomic pseudopotentials suitable for our system

In [36]:
print( os.getcwd())

/projects/academic/alexeyak/Software/Libra/notebooks/Example9_NAMD/step2a


Assume we have already produced a QE MD trajectory and it is stored in the file **x0.md.out** (which we copied in the present directory).

We need to also create:

* a **x0.scf.in** file that contains the parameters for QE calculations (the type of calculation should be *scf*). The file should not contain the atomic coordiantes section, but should contain the cell parameters sections or occupations if they are used. 

* a **x0.exp.in** file (also to be placed in the present directory). It shall describe the procedured for the wavefunction "export" operation - mainly the location and names of atomic pseudopotentials and the correct prefix for the files. 

In the section below, the user can define (e.g. via copy/paste) the content of the corresponding files and the files will be automatically generated by Python

In [37]:
PP_dir = os.getcwd()+"/PP/"

scf_in = """&CONTROL
  calculation = 'scf',
  dt = 20.67055,
  nstep = 10,
  pseudo_dir = '%s',
  outdir = './',
  prefix = 'x0',
  disk_io = 'low',
  wf_collect = .true.
/

&SYSTEM
  ibrav = 0,
  celldm(1) = 1.89,
  nat = 4,
  ntyp = 2,
  nspin = 2,
  starting_magnetization(1) = 0.1,
  nbnd = 40,
  ecutwfc = 40,
  tot_charge = 0.0,
  occupations = 'smearing',
  smearing = 'gaussian',
  degauss = 0.005,
  nosym = .true.,
/

&ELECTRONS
  electron_maxstep = 300,
  conv_thr = 1.D-5,
  mixing_beta = 0.45,
/

&IONS
  ion_dynamics = 'verlet',
  ion_temperature = 'andersen',
  tempw = 300.00 ,
  nraise = 1,
/


ATOMIC_SPECIES
 Cd 121.411  Cd.pbe-n-rrkjus_psl.1.0.0.UPF
 Se 78.96    Se.pbe-dn-rrkjus_psl.1.0.0.UPF


K_POINTS automatic
 1 1 1 0 0 0

CELL_PARAMETERS (alat=  1.89000000)
   4.716986504  -0.015512615  -0.002400656
  -2.371926710   4.062829845  -0.000273730
  -0.002552594  -0.001387965   8.436361230

""" % (PP_dir)

exp_in = """&inputpp
  prefix = 'x0',
  outdir = './',
  pseudo_dir = '%s',
  psfile(1) = 'Cd.pbe-n-rrkjus_psl.1.0.0.UPF',
  psfile(2) = 'Se.pbe-dn-rrkjus_psl.1.0.0.UPF',
  single_file = .FALSE.,
  ascii = .TRUE.,
  uspp_spsi = .FALSE.,
/

""" % (PP_dir)

f = open("x0.scf.in", "w")
f.write(scf_in)
f.close()

f = open("x0.exp.in", "w")
f.write(exp_in)
f.close()


#print scf_in
#print exp_in

The following section will clean up the previous results and temporary directory (BEWARE!!! you may not always want to do this for it will delete expensive results)

In [38]:
# Remove the previous results and temporary working directory from the previous runs
os.system("rm -r res")
os.system("rm -r wd")

# Create the new results directory
os.system("mkdir res")
rd = os.getcwd()+"/res"          # where all the results stuff will go

Now we need to setup the submit script template - it will be used to create actual submit scripts (by substituting the param1 and param2 variables) in the working directory. Then those files will be distributed among the job directories and will be used to submit the actual jobs.

This section also defines the parameters to be used by the **step2.run()** (and other functions called inside, so look for the description all all suitable parameters). The meaning of the most parameters is quite intuitive. Let me just clarify a couple less-obvious points. 

That section of the code looks weird because:
* it is a Python sctring that defines ...
* a SLURM script that uses bash commands and calls ..
* the Python script to be executed, which eventually calls ...
* the Libra modules to do the step2 calculations

In this example, I plan to submit the calculation in the HPC cluster to "parallelize" the calculations via the SLURM batch system, so specify "BATCH_SYSTEM":"srun"

The system in this example has 56 electrons, so the HOMO would correspond to number 28 and LUMO to number 29. In this example, we use more orbitals - those below HOMO and above LUMO: minband = 20 and maxband = 39, so we could study the HOMO-LUMO transitions as well as all other types of relaxation. In all, there are 20 orbitals included in our present active space. The resulting files will contain matrixes 40 by 40 - because we plot alpha and beta channels. Just a reminder: the orbital indexing starts from 1.

In [39]:
submit_str = """#!/bin/sh
#SBATCH --partition=valhalla --qos=valhalla
#SBATCH --clusters=chemistry
#SBATCH --time=02:00:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=4
#SBATCH --mem=5000
#SBATCH --mail-user=alexeyak@buffalo.edu

echo "SLURM_JOBID="$SLURM_JOBID
echo "SLURM_JOB_NODELIST="$SLURM_JOB_NODELIST
echo "SLURM_NNODES="$SLURM_NNODES
echo "SLURMTMPDIR="$SLURMTMPDIR
echo "working directory="$SLURM_SUBMIT_DIR

NPROCS=`srun --nodes=${SLURM_NNODES} bash -c 'hostname' |wc -l`
echo NPROCS=$NPROCS

module load intel/16.0
module load intel-mpi/5.1.1
module load mkl/11.3
module load espresso


#The PMI library is necessary for srun
export I_MPI_PMI_LIBRARY=/usr/lib64/libpmi.so

# These will be assigned automatically, leave them as they are
param1=
param2=

# This is invocation of the scripts which will further handle NA-MD calclculations on the NAC calculation step
# NOTE: minband - starting from 1
#       maxband - is included

python -c \"from libra_py.workflows.nbra import step2
params = {}
params[\\"EXE\\"] = \\"pw.x\\" 
params[\\"EXE_EXPORT\\"] = \\"pw_export.x\\"
params[\\"BATCH_SYSTEM\\"] = \\"srun\\"
params[\\"NP\\"] = 4
params[\\"start_indx\\"] = $param1
params[\\"stop_indx\\"] = $param2
params[\\"dt\\"] = %8.5f
params[\\"prefix0\\"] = \\"x0.scf\\" 
params[\\"nac_method\\"] = 1
params[\\"minband\\"] = 20
params[\\"maxband\\"] = 39
params[\\"minband_soc\\"] = 20
params[\\"maxband_soc\\"] = 39
params[\\"compute_Hprime\\"] = True
params[\\"wd\\"] = \\"wd\\"
params[\\"rd\\"] = \\"%s\\"
params[\\"verbosity\\"] = 0
step2.run(params)
\"

""" % ( 1.0*units.fs2au, os.getcwd()+"/res" )


f = open("submit_templ.slm", "w")
f.write(submit_str)
f.close()

#print submit_str

We'll use **QE_methods.out2inp()** function to convert the MD trajectory into a bunch of input files for SCF calculation - this is something we'll need for NAC calculations. 

In this case, you need to setup the *iinit* and *ifinal* variables which determine which steps of the original MD trajectory will be used to produce the input files and subsequenctly used in the NACs calculations

All these files will be generated in the temporarily-created **wd** directory. The system then "cd" into that directory to start the consecutive operations in that directory.

In [40]:
iinit = 0
ifinal = 10

QE_methods.out2inp("x0.md.out","x0.scf.in","wd","x0.scf", iinit, ifinal,1)

os.system("cp submit_templ.slm wd")
os.system("cp x0.exp.in wd") 

0

Now, lets change into our working (temporary) directory, copy all the template files and submit the calculations of multiple jobs. Come back to the original working directory once we are done.

In [41]:
os.chdir("wd")

tot_nsteps = 10
nsteps_per_job = 5
hpc_utils.distribute(0,tot_nsteps,nsteps_per_job,"submit_templ.slm",["x0.exp.in"],["x0.scf"],2)

os.chdir("../")

In [42]:
print( os.getcwd())

/projects/academic/alexeyak/Software/Libra/notebooks/Example9_NAMD/step2a
